In [1]:
import os

def import_from_directory(directory, module_name):
    current_dir = os.getcwd()
    os.chdir(directory)
    module = __import__(module_name)
    os.chdir(current_dir)
    return module

test_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'test_base')
model_base = import_from_directory('/home/jovyan/work/spectrum/notebooks/src', 'model_base')

2024-05-07 05:04:12.873053: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 05:04:12.910100: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-07 05:04:12.910130: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-07 05:04:12.911188: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-07 05:04:12.917670: I tensorflow/core/platform/cpu_feature_guar

In [2]:
SEED = 42
model_base.set_seed(SEED)

desired_directory = '/home/jovyan/work/spectrum/notebooks/'
os.chdir(desired_directory)

In [3]:
import pandas as pd

In [17]:
df = pd.read_csv('Data/test_data/rikenlipid_data.csv')
df['datasetID'] = df['dataset'].str.split('_').str[0]
df_pos = df[df['Adducttype'].str.contains('\]\+')  |  df['Adducttype'].str.contains('\]2\+')].reset_index(drop=True)
df_pos = df_pos[~df_pos['MSMSspectrum'].isna()]

In [18]:
df_pos2 = model_base.cal_df_sel_column(df_pos).drop_duplicates().reset_index(drop=True)
df_pos_wide = model_base.cal_wide_df(df_pos2)
df_pos_wide['Ontology'] = df_pos_wide['Ontology'].str.replace(r'-', '_')

In [19]:
column_path = 'Data/model_data/pos/posmodelcolumn_all.pkl'
replacement_dict_path = 'Data/model_data/pos/pos_replacement_dict_all.pkl'
model_path = "models/pos_class_model"

pred1st, predcandidate, num, df_test_predclass = test_base.cal_accscore(df_pos_wide, replacement_dict_path, column_path, model_path)

259/259 [==============================] - 2s 6ms/step


In [20]:
pred1st1, predcandidate1, num1, df_test_predclass1 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, (1, 11), type='btw', title = 'test Accuracy in Different curator')
pred1st2, predcandidate2, num2, df_test_predclass2 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, [78],  title = 'test Accuracy in Waters')
pred1st3, predcandidate3, num3, df_test_predclass3 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, [79],  title = 'test Accuracy in Thermo')
pred1st4, predcandidate4, num4, df_test_predclass4 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, [80],  title = 'test Accuracy in Agilent')
pred1st5, predcandidate5, num5, df_test_predclass5 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, [83, 84],  title = 'test Accuracy in Bruker')
pred1st6, predcandidate6, num6, df_test_predclass6 = test_base.make_pred_summary(df_pos_wide, replacement_dict_path, column_path, model_path, [81],  title = 'test Accuracy in Sciex 6600-SWATH')

test Accuracy in Different curator
115/115 [==============================] - 1s 6ms/step
1st pred : 3432
candidate : 3599
total: 3669
----
test Accuracy in Waters
3/3 [==============================] - 0s 7ms/step
1st pred : 82
candidate : 84
total: 88
----
test Accuracy in Thermo
6/6 [==============================] - 0s 6ms/step
1st pred : 160
candidate : 165
total: 165
----
test Accuracy in Agilent
8/8 [==============================] - 0s 6ms/step
1st pred : 220
candidate : 235
total: 239
----
test Accuracy in Bruker
11/11 [==============================] - 0s 6ms/step
1st pred : 301
candidate : 325
total: 343
----
test Accuracy in Sciex 6600-SWATH
8/8 [==============================] - 0s 7ms/step
1st pred : 205
candidate : 228
total: 241
----


In [22]:
data = [['Different curator(Sciex TripleTOF 6600)', num1, pred1st1, predcandidate1-pred1st1],
        ['Waters XevoG2 QTOF', num2, pred1st2, predcandidate2-pred1st2],
        ['Thermo Q Exactive Plus', num3, pred1st3, predcandidate3-pred1st3],
        ['Agilent 6546 QTOF', num4, pred1st4, predcandidate4-pred1st4],
        ['Bruker timsTOF Pro', num5, pred1st5, predcandidate5-pred1st5],
        ['Sciex TripleTOF 6600(SWATH)', num6, pred1st6, predcandidate6-pred1st6]]

df_predresult = pd.DataFrame(data, columns=['Name', 'Sample number', '1st pred', 'Candidate'])
df_predresult['Non predicted'] = df_predresult['Sample number']-(df_predresult['1st pred']+df_predresult['Candidate'])

df_predresult.to_csv('Data/test_data/pos/rikenlipid_df_pos_model_pred_result.csv', index=False)